In [1]:
import numpy as np
import xgboost as xgb
from wideboost.wrappers.wxgb import wxgbModel
from wideboost.wrappers import wxgb

import tensorflow as tf
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

ds_train, ds_info = tfds.load(
    'forest_fires',
    split=['train'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

for i in ds_train[0].batch(60000):
    a = i

In [2]:
# Keep numeric columns
X = []
for s in a[1].keys():
    if a[1][s].dtype != tf.string:
        X.append(a[1][s].numpy().reshape([-1,1]))

X = np.concatenate(X,axis=1)

idx = np.arange(X.shape[0])
np.random.seed(1)
train_idx = np.random.choice(idx, round(0.7*X.shape[0]), replace=False)
test_idx = np.setdiff1d(idx,train_idx)

xtrain = X[train_idx,:]
ytrain = a[0].numpy()[train_idx]

xtest = X[test_idx,:]
ytest = a[0].numpy()[test_idx]

dtrain = xgb.DMatrix(xtrain,label=ytrain)
dtest = xgb.DMatrix(xtest,label=ytest)

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval

best_val = np.inf

def objective(param):
    global best_val
    watchlist = [(dtrain,'train'),(dtest,'test')]
    ed1_results = dict()
    print(param)
    wbst = wxgb.train(param, dtrain, 100,watchlist,evals_result=ed1_results)
    
    output = min(ed1_results['test']['rmse'])
    
    if output < best_val:
        print("NEW BEST VALUE!")
        best_val = output
    
    return {'loss': output, 'status': STATUS_OK }

spc = {
    'btype': hp.choice('btype',['I']),
    'extra_dims': hp.choice('extra_dims',[0]),
    'objective': hp.choice('objective',['reg:squarederror']),
    'eval_metric':hp.choice('eval_metric',[['rmse']]),
    'eta': hp.loguniform('eta', -7, 0),
    'max_depth' : hp.choice('max_depth',range(1,11)),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1),
    'min_child_weight': hp.loguniform('min_child_weight', -16, 5),
    'alpha': hp.choice('alpha', [0, hp.loguniform('alpha_positive', -16, 2)]),
    'lambda': hp.choice('lambda', [0, hp.loguniform('lambda_positive', -16, 2)]),
    'gamma': hp.choice('gamma', [0, hp.loguniform('gamma_positive', -16, 2)])
}

best = fmin(objective,
    space=spc,
    algo=tpe.suggest,
    max_evals=100)



{'alpha': 0, 'btype': 'I', 'colsample_bylevel': 0.5361306505252079, 'colsample_bytree': 0.6589339719669678, 'eta': 0.3871085405413005, 'eval_metric': ('rmse',), 'extra_dims': 0, 'gamma': 0, 'lambda': 3.1668209633171666e-07, 'max_depth': 2, 'min_child_weight': 0.006094610755595525, 'objective': 'reg:squarederror', 'subsample': 0.6751692074059168}
Overwriting param `num_class`                          
Overwriting param `objective` while setting `obj` in train.
Taking first argument of eval_metric. Multiple evals not supported using xgboost backend.
Moving param `eval_metric` to an feval.                
Setting param `disable_default_eval_metric` to 1.      
[12:30:09] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { btype } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you

[4]	train-rmse:1.98540	test-rmse:50.29075                               

[5]	train-rmse:1.45102	test-rmse:50.55567                               

[6]	train-rmse:0.95534	test-rmse:50.66324                               

[7]	train-rmse:0.74885	test-rmse:50.68054                               

[8]	train-rmse:0.53229	test-rmse:50.69989                               

[9]	train-rmse:0.43955	test-rmse:50.75710                               

[10]	train-rmse:0.31841	test-rmse:50.76192                              

[11]	train-rmse:0.24470	test-rmse:50.77936                              

[12]	train-rmse:0.21518	test-rmse:50.77705                              

[13]	train-rmse:0.18913	test-rmse:50.77840                              

[14]	train-rmse:0.17201	test-rmse:50.77416                              

[15]	train-rmse:0.16865	test-rmse:50.77504                              

[16]	train-rmse:0.16366	test-rmse:50.77018                              

[17]	train-rmse:0.16171	test-rmse:50.7

[0]	train-rmse:51.68219	test-rmse:26.80372                              

[1]	train-rmse:37.68958	test-rmse:27.80155                              

[2]	train-rmse:34.70756	test-rmse:28.20963                              

[3]	train-rmse:26.50245	test-rmse:33.86307                              

[4]	train-rmse:20.84662	test-rmse:34.66909                              

[5]	train-rmse:19.69694	test-rmse:34.73210                              

[6]	train-rmse:16.39670	test-rmse:35.12888                              

[7]	train-rmse:14.43379	test-rmse:35.19944                              

[8]	train-rmse:13.57062	test-rmse:35.28408                              

[9]	train-rmse:12.08105	test-rmse:35.66152                              

[10]	train-rmse:11.46788	test-rmse:35.61993                             

[11]	train-rmse:10.41421	test-rmse:36.51199                             

[12]	train-rmse:10.08366	test-rmse:36.91612                             

[13]	train-rmse:9.66140	test-rmse:36.7

Setting param `disable_default_eval_metric` to 1.                       
[12:30:12] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { btype } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:75.19803	test-rmse:27.10627                              

[1]	train-rmse:74.96930	test-rmse:26.87346                              

[2]	train-rmse:74.87329	test-rmse:26.75320                              

[3]	train-rmse:74.64452	test-rmse:26.57254                              

[4]	train-rmse:74.52973	test-rmse:26.45789                              

[5]	train-rmse:74.42343	test-rmse:26.40100                              

[6]	train-rmse:74.30777	test-rmse:26.31671                              

[7]	train-rmse:74.15611	test-rmse:26.30383            

Overwriting param `num_class`                                           
Overwriting param `objective` while setting `obj` in train.             
Taking first argument of eval_metric. Multiple evals not supported using xgboost backend.
Moving param `eval_metric` to an feval.                                 
Setting param `disable_default_eval_metric` to 1.                       
[12:30:12] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { btype } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:75.42088	test-rmse:27.33635                              

[1]	train-rmse:75.39172	test-rmse:27.30486                              

[2]	train-rmse:75.38037	test-rmse:27.29440                              

[3]	train-rmse:75.35358	test-rmse:27.2649

...

[17]	train-rmse:59.79964	test-rmse:26.99546                              

[18]	train-rmse:59.28518	test-rmse:27.00264                              

[19]	train-rmse:58.48418	test-rmse:27.00995                              

[20]	train-rmse:57.80920	test-rmse:27.02774                              

[21]	train-rmse:57.45146	test-rmse:27.10722                              

[22]	train-rmse:56.69148	test-rmse:27.10092                              

[23]	train-rmse:56.38907	test-rmse:27.13753                              

[24]	train-rmse:55.65896	test-rmse:27.15809                              

[25]	train-rmse:55.03718	test-rmse:27.19224                              

[26]	train-rmse:54.42704	test-rmse:27.21967                              

[27]	train-rmse:53.84602	test-rmse:27.26143                              

[28]	train-rmse:53.17285	test-rmse:27.24577                              

[29]	train-rmse:52.65791	test-rmse:27.23536                              

[30]	train-rmse:52.37432	

[11]	train-rmse:2.74841	test-rmse:39.12595                               

[12]	train-rmse:2.32621	test-rmse:39.45135                               

[13]	train-rmse:2.11007	test-rmse:39.42555                               

[14]	train-rmse:1.67607	test-rmse:39.43349                               

[15]	train-rmse:1.36749	test-rmse:39.47572                               

[16]	train-rmse:1.14492	test-rmse:39.51256                               

[17]	train-rmse:1.03743	test-rmse:39.49838                               

[18]	train-rmse:0.91141	test-rmse:39.51584                               

[19]	train-rmse:0.83531	test-rmse:39.51040                               

[20]	train-rmse:0.80425	test-rmse:39.50629                               

[21]	train-rmse:0.67716	test-rmse:39.51663                               

[22]	train-rmse:0.65722	test-rmse:39.52673                               

[23]	train-rmse:0.60010	test-rmse:39.56405                               

[24]	train-rmse:0.56487	t

[5]	train-rmse:31.24000	test-rmse:38.29086                               

[6]	train-rmse:26.37519	test-rmse:47.37308                               

[7]	train-rmse:25.23443	test-rmse:49.07882                               

[8]	train-rmse:24.24224	test-rmse:49.29189                               

[9]	train-rmse:20.38784	test-rmse:51.40608                               

[10]	train-rmse:18.13882	test-rmse:51.76046                              

[11]	train-rmse:17.66277	test-rmse:52.14591                              

[12]	train-rmse:15.54347	test-rmse:55.78002                              

[13]	train-rmse:14.34568	test-rmse:55.68358                              

[14]	train-rmse:13.59555	test-rmse:55.89630                              

[15]	train-rmse:13.14057	test-rmse:55.91570                              

[16]	train-rmse:12.58685	test-rmse:55.83970                              

[17]	train-rmse:12.00272	test-rmse:55.77625                              

[18]	train-rmse:10.72090	

[0]	train-rmse:94.36036	test-rmse:68.37174                               

[1]	train-rmse:91.87675	test-rmse:108.88916                              

[2]	train-rmse:81.82953	test-rmse:112.77614                              

[3]	train-rmse:75.31179	test-rmse:108.68989                              

[4]	train-rmse:52.47577	test-rmse:104.56439                              

[5]	train-rmse:53.23110	test-rmse:105.67306                              

[6]	train-rmse:49.30722	test-rmse:109.15893                              

[7]	train-rmse:47.56271	test-rmse:109.96433                              

[8]	train-rmse:45.14863	test-rmse:111.00487                              

[9]	train-rmse:41.89168	test-rmse:114.87448                              

[10]	train-rmse:40.53250	test-rmse:115.85103                             

[11]	train-rmse:40.39742	test-rmse:114.54297                             

[12]	train-rmse:40.01398	test-rmse:115.69674                             

[13]	train-rmse:39.49989	

Setting param `disable_default_eval_metric` to 1.                        
[12:31:49] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { btype } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:47.33391	test-rmse:35.09580                               

[1]	train-rmse:49.73282	test-rmse:42.51429                               

[2]	train-rmse:35.93877	test-rmse:41.89096                               

[3]	train-rmse:35.12012	test-rmse:43.11994                               

[4]	train-rmse:11.48680	test-rmse:43.45206                               

[5]	train-rmse:6.56665	test-rmse:43.86881                                

[6]	train-rmse:5.33805	test-rmse:45.24033                                

[7]	train-rmse:4.56987	test-rmse:45.72129     

In [4]:
print(best_val)
print(space_eval(spc, best))

25.451185
{'alpha': 0, 'btype': 'I', 'colsample_bylevel': 0.9509209250400708, 'colsample_bytree': 0.7082745692450216, 'eta': 0.49646774963918355, 'eval_metric': ('rmse',), 'extra_dims': 0, 'gamma': 0.05675964174839698, 'lambda': 0, 'max_depth': 3, 'min_child_weight': 0.0003603428234530497, 'objective': 'reg:squarederror', 'subsample': 0.6454324200394408}


In [ ]:
# Trial 1
# 36.809885

# 25.451185
# {'alpha': 0, 'btype': 'I', 'colsample_bylevel': 0.9509209250400708, 'colsample_bytree': 0.7082745692450216, 'eta': 0.49646774963918355, 'eval_metric': ('rmse',), 'extra_dims': 0, 'gamma': 0.05675964174839698, 'lambda': 0, 'max_depth': 3, 'min_child_weight': 0.0003603428234530497, 'objective': 'reg:squarederror', 'subsample': 0.6454324200394408}

# Trial 3
# 59.884377
#{'alpha': 0, 'btype': 'I', 'colsample_bylevel': 0.8778935246905272, 'colsample_bytree': 0.631779674395855, 'eta': 0.3837695511757347, 'eval_metric': ('rmse',), 'extra_dims': 0, 'gamma': 0, 'lambda': 0.36948618470662503, 'max_depth': 3, 'min_child_weight': 3.614725012825161e-06, 'objective': 'reg:squarederror', 'subsample': 0.5596549639505122}

# Trial 4
# 80.607321
# {'alpha': 0, 'btype': 'I', 'colsample_bylevel': 0.5811385996129368, 'colsample_bytree': 0.7876123744455882, 'eta': 0.853742343152421, 'eval_metric': ('rmse',), 'extra_dims': 0, 'gamma': 0, 'lambda': 0.014279589826310397, 'max_depth': 4, 'min_child_weight': 0.001143436734111359, 'objective': 'reg:squarederror', 'subsample': 0.8510036127464964}

# Trial 5
# 86.268094
# {'alpha': 5.605849047937274e-07, 'btype': 'I', 'colsample_bylevel': 0.995162502533999, 'colsample_bytree': 0.880457738869928, 'eta': 0.9527860187181443, 'eval_metric': ('rmse',), 'extra_dims': 0, 'gamma': 0.026675466059276922, 'lambda': 0.0007669691342082616, 'max_depth': 3, 'min_child_weight': 0.9005752353106548, 'objective': 'reg:squarederror', 'subsample': 0.5533089593791273}